In [443]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gurobipy as gp
from gurobipy import GRB
import os
from itertools import product
from functions import (load_parameters, load_generation_data, load_price_data, generate_randomized_generation,
generate_rt_scenarios, plot_generation_data, plot_randomized_generation, plot_scenarios_for_generator, plot_rt_scenarios)

generation_data, I, T = load_generation_data(date_filter="2022-07-18")
S, R, P_RT_v, K, K0, M1, M2 = load_parameters(I, T, generation_data)
P_DA, P_PN = load_price_data()

✅ 총 5개 파일을 불러왔습니다: 1201.csv, 137.csv, 401.csv, 524.csv, 89.csv
📊 데이터 Shape: I=5, T=24, S=5
✅ 시뮬레이션 초기화 완료: S=5, Randomness='high', M1=618.95, M2=1584.34


/Users/jangseohyun/Documents/workspace/symply/DER/functions.py:96: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  nyc_rt["Hour"] = nyc_rt["Time Stamp"].dt.floor("H")


In [444]:
# models
only = gp.Model("only")
only.Params.OutputFlag = 0
only.setParam("MIPGap", 1e-7)

x = only.addVars(I, T, vtype=GRB.CONTINUOUS, lb=0, name="x")
yp = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_plus")
ym = only.addVars(I, T, S, vtype=GRB.CONTINUOUS, lb=0, name="y_minus")
zeta = only.addVars(I, T, S, vtype=GRB.BINARY, name="z")
only.update()

obj = gp.quicksum(P_DA[t] * x[i, t] for i in range(I) for t in range(T)) + gp.quicksum(
    1/S * (P_RT_v[t, s] * yp[i, t, s] - P_PN[t] * ym[i, t, s]) for i in range(I) for t in range(T) for s in range(S))
only.setObjective(obj, GRB.MAXIMIZE)

for i in range(I):
    for t in range(T):
        for s in range(S):
            only.addConstr(R[i, t, s] - x[i, t] == yp[i, t, s] - ym[i, t, s])
            only.addConstr(yp[i, t, s] <= R[i, t, s])
            only.addConstr(yp[i, t, s] <= M1 * zeta[i, t, s])
            only.addConstr(ym[i, t, s] <= M1 * (1 - zeta[i, t, s]))

only.optimize()

if only.status == GRB.OPTIMAL:
    print(f"[ONLY] {only.objVal}")
else:
    print("No optimal solution found.")
    
# ----------------------------------

agg = gp.Model("agg")
agg.Params.OutputFlag = 0
agg.setParam("MIPGap", 1e-7)

a = agg.addVars(T, vtype=GRB.CONTINUOUS, lb=0, name="alpha")
bp = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_plus")
bm = agg.addVars(T, S, vtype=GRB.CONTINUOUS, lb=0, name="beta_minus")
mu = agg.addVars(T, S, vtype=GRB.BINARY, name="z")
agg.update()

obj = gp.quicksum(P_DA[t] * a[t] for t in range(T)) + gp.quicksum(
    1/S * (P_RT_v[t, s] * bp[t, s] - P_PN[t] * bm[t, s]) for t in range(T) for s in range(S))
agg.setObjective(obj, GRB.MAXIMIZE)

for t in range(T):
    for s in range(S):
        agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) - a[t] == bp[t, s] - bm[t, s])
        agg.addConstr(gp.quicksum(R[i, t, s] for i in range(I)) >= bp[t, s])
        agg.addConstr(bp[t, s] <= M2 * mu[t, s])
        agg.addConstr(bm[t, s] <= M2 * (1 - mu[t, s]))
agg.optimize()

if agg.status == GRB.OPTIMAL:
    print(f"[AGG] {agg.objVal}")
else:
    print("No optimal solution found.")

[ONLY] 1570162.4925295592
[AGG] 1593301.2853834806


In [445]:
x_v   = np.array([[only.getVarByName(f"x[{i},{t}]").X for t in range(T)] for i in range(I)])
yp_v = np.array([[[only.getVarByName(f"y_plus[{i},{t},{s}]").X for s in range(S)] for t in range(T)] for i in range(I)])
ym_v = np.array([[[only.getVarByName(f"y_minus[{i},{t},{s}]").X for s in range(S)] for t in range(T)] for i in range(I)])
a_v  = np.array([agg.getVarByName(f"alpha[{t}]").X for t in range(T)])
bp_v = np.array([[agg.getVarByName(f"beta_plus[{t},{s}]").X for s in range(S)] for t in range(T)])
bm_v = np.array([[agg.getVarByName(f"beta_minus[{t},{s}]").X for s in range(S)] for t in range(T)])

idx = 4

xv   = x_v
ypv  = yp_v[:, :, idx]
ymv  = ym_v[:, :, idx]
av   = a_v
bpv  = bp_v[:, idx]
bmv  = bm_v[:, idx]
P_RT = P_RT_v[:, idx]

In [446]:
delta_DA = np.zeros(T)
delta_BMp = np.zeros(T)
delta_BMm = np.zeros(T)
lambd = np.zeros((I, T))
total_surplus = np.zeros(T)

for t in range(T):
    delta_DA[t]  = av[t]  - np.sum(xv[:, t])
    delta_BMp[t] = bpv[t] - np.sum(ypv[:, t])
    delta_BMm[t] = bmv[t] - np.sum(ymv[:, t])
    total_surplus[t]   = P_DA[t] * delta_DA[t] + P_RT[t] * delta_BMp[t] - P_PN[t] * delta_BMm[t]

print(f"{'t':>2} | {'ΔDA':>10} | {'ΔBM+':>12} | {'ΔBM-':>12} || {'Surplus':>10}")
print("-" * 60)
for t in range(T):
    print(f"{t:2} | {delta_DA[t]:10.4f} | {delta_BMp[t]:12.4f} | {delta_BMm[t]:12.4f} || {total_surplus[t]:10.4f}")
print("-" * 60)
print(f"{'Σ':>2} | {'':>10} | {'':>12} | {'':>12} || {total_surplus.sum():10.4f}")

 t |        ΔDA |         ΔBM+ |         ΔBM- ||    Surplus
------------------------------------------------------------
 0 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 1 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 2 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 3 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 4 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 5 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 6 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 7 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
 8 |    11.8745 |       0.0000 |      11.8745 ||  -673.2799
 9 |    14.7159 |     -14.7159 |       0.0000 ||   283.7545
10 |    63.5550 |     -99.0835 |     -35.5285 ||  1816.4389
11 |   138.8316 |    -138.8316 |       0.0000 || -2815.0258
12 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
13 |     0.0000 |       0.0000 |       0.0000 ||     0.0000
14 |   393.3783 |    -459.4262 |     -6

In [447]:
Po        = xv + ypv - ymv                         
Po_prime  = xv + ypv - ymv
delta_Po  = Po - Po_prime
delta_Pop = np.where(delta_Po > 0, delta_Po, 0)
delta_Pom = np.where(delta_Po < 0, -delta_Po, 0)

varphi_p = ypv + delta_Pop
varphi_m = ymv + delta_Pom

print(f"{'i':>2} {'t':>2} | {'y⁺':>8} | {'ΔP⁺':>8} | {'φ⁺':>8} || {'y⁻':>8} | {'ΔP⁻':>8} | {'φ⁻':>8}")
print("-" * 75)
for i in range(I):
    for t in range(T):
        print(f"{i:2} {t:2} | {ypv[i,t]:8.2f} | {delta_Pop[i,t]:8.2f} | {varphi_p[i,t]:8.2f} || {ymv[i,t]:8.2f} | {delta_Pom[i,t]:8.2f} | {varphi_m[i,t]:8.2f}")

 i  t |       y⁺ |      ΔP⁺ |       φ⁺ ||       y⁻ |      ΔP⁻ |       φ⁻
---------------------------------------------------------------------------
 0  0 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  1 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  2 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  3 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  4 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  5 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  6 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  7 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0  8 |     0.00 |     0.00 |     0.00 ||     5.00 |     0.00 |     5.00
 0  9 |     0.00 |     0.00 |     0.00 ||     0.00 |     0.00 |     0.00
 0 10 |    16.35 |     0.00 |    16.35 ||     0.00 |     0.00 |     0.00
 0 11 |    65.42 |     0.00 |    65.42 ||     0.

In [448]:
delta_DAp = np.where(delta_DA > 0,  delta_DA, 0)
delta_DAm = np.where(delta_DA < 0, -delta_DA, 0)
print(f"{'t':>2} | {'α':>6} | {'∑x':>6} | {'ΔDA':>8} | {'ΔDA+':>8} | {'ΔDA-':>8}")
print("-" * 55)
for t in range(T):
    print(f"{t:2} | {av[t]:6.2f} | {np.sum(xv[:,t]):6.2f} | {delta_DA[t]:8.2f} | {delta_DAp[t]:8.2f} | {delta_DAm[t]:8.2f}")

 t |      α |     ∑x |      ΔDA |     ΔDA+ |     ΔDA-
-------------------------------------------------------
 0 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 1 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 2 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 3 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 4 |   0.66 |   0.66 |     0.00 |     0.00 |     0.00
 5 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 6 |   2.26 |   2.26 |     0.00 |     0.00 |     0.00
 7 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
 8 |  41.36 |  29.48 |    11.87 |    11.87 |     0.00
 9 | 118.37 | 103.65 |    14.72 |    14.72 |     0.00
10 | 449.08 | 385.53 |    63.56 |    63.56 |     0.00
11 | 373.93 | 235.10 |   138.83 |   138.83 |     0.00
12 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
13 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
14 | 1304.47 | 911.09 |   393.38 |   393.38 |     0.00
15 |   0.00 |   0.00 |     0.00 |     0.00 |     0.00
16 | 552.14 | 386.39 |   

In [449]:
E_INp = np.zeros(T)
E_INm = np.zeros(T)

for t in range(T):
    E_INp[t] = np.sum(varphi_p[:, t]) - (bpv[t] + delta_DAp[t])
    E_INm[t] = np.sum(varphi_m[:, t]) - (bmv[t] + delta_DAm[t])
    
print(f"{'t':>2} | {'∑φ⁺':>9} | {'β⁺':>9} | {'ΔDA+':>7} | {'E_IN⁺':>8} || {'∑φ⁻':>7} | {'β⁻':>6} | {'ΔDA-':>6} | {'E_IN⁻':>7}")
print("-" * 90)
for t in range(T):
    sum_phi_p = np.sum(varphi_p[:, t])
    sum_phi_m = np.sum(varphi_m[:, t])
    print(f"{t:2} | {sum_phi_p:9.2f} | {bpv[t]:9.2f} | {delta_DAp[t]:7.2f} | {E_INp[t]:8.2f} ||"
          f" {sum_phi_m:7.2f} | {bmv[t]:6.2f} | {delta_DAm[t]:6.2f} | {E_INm[t]:7.2f}")

 t |       ∑φ⁺ |        β⁺ |    ΔDA+ |    E_IN⁺ ||     ∑φ⁻ |     β⁻ |   ΔDA- |   E_IN⁻
------------------------------------------------------------------------------------------
 0 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 1 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 2 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 3 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 4 |      0.13 |      0.13 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 5 |      0.00 |      0.00 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 6 |      2.43 |      2.43 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 7 |     20.48 |     20.48 |    0.00 |     0.00 ||    0.00 |   0.00 |   0.00 |    0.00
 8 |      0.00 |      0.00 |   11.87 |   -11.87 ||    6.28 |  18.15 |   0.00 |  -11.87
 9 |     93.67 |     78.95 |   14.72 | 

In [450]:
m = 1e-16
e_INp = np.zeros((I, T))
e_INm = np.zeros((I, T))

for t in range(T):
    for i in range(I):
        e_INp[i, t] = (varphi_p[i, t] / (np.sum(varphi_p[:, t]) + m)) * E_INp[t]
        e_INm[i, t] = (varphi_m[i, t] / (np.sum(varphi_m[:, t]) + m)) * E_INm[t]

print(f"{'i':>2} {'t':>2} | {'φ⁺':>8} | {'∑φ⁺':>8} | {'e_IN⁺':>10} || {'φ⁻':>8} | {'∑φ⁻':>8} | {'e_IN⁻':>10}")
print("-" * 80)
for t in range(20, 23):
    for i in range(I):
        sum_phi_p = np.sum(varphi_p[:, t])
        sum_phi_m = np.sum(varphi_m[:, t])
        print(f"{i:2} {t:2} | {varphi_p[i, t]:8.2f} | {sum_phi_p:8.2f} | {e_INp[i, t]:10.2f} ||"
              f" {varphi_m[i, t]:8.2f} | {sum_phi_m:8.2f} | {e_INm[i, t]:10.2f}")

 i  t |       φ⁺ |      ∑φ⁺ |      e_IN⁺ ||       φ⁻ |      ∑φ⁻ |      e_IN⁻
--------------------------------------------------------------------------------
 0 20 |    34.62 |    86.57 |       3.87 ||     0.00 |     9.67 |       0.00
 1 20 |     3.24 |    86.57 |       0.36 ||     0.00 |     9.67 |       0.00
 2 20 |     0.00 |    86.57 |       0.00 ||     9.67 |     9.67 |       9.67
 3 20 |    43.30 |    86.57 |       4.84 ||     0.00 |     9.67 |       0.00
 4 20 |     5.42 |    86.57 |       0.61 ||     0.00 |     9.67 |       0.00
 0 21 |     4.56 |     4.64 |       0.48 ||     0.00 |     0.49 |       0.00
 1 21 |     0.00 |     4.64 |       0.00 ||     0.00 |     0.49 |       0.00
 2 21 |     0.00 |     4.64 |       0.00 ||     0.49 |     0.49 |       0.49
 3 21 |     0.08 |     4.64 |       0.01 ||     0.00 |     0.49 |       0.00
 4 21 |     0.00 |     4.64 |       0.00 ||     0.00 |     0.49 |       0.00
 0 22 |     0.00 |     0.00 |       0.00 ||     0.00 |     0.00 |       

In [451]:
delta_INp = np.zeros(T)
delta_INm = np.zeros(T)
Lambda_IN = np.zeros(T)
rho_IN = np.zeros(T)

for t in range(T):
    
    if delta_BMp[t] < 0 and E_INp[t] > 1e-6:
        delta_INp[t] = min(E_INp[t], -delta_BMp[t])
    else:
        delta_INp[t] = 0
    
    if delta_BMm[t] < 0 and E_INm[t] > 1e-6:
        delta_INm[t] = min(E_INm[t], -delta_BMm[t])
    else:
        delta_INm[t] = 0
        
    Lambda_IN[t] = P_RT[t] * delta_INp[t] - P_PN[t] * delta_INm[t]
    rho_IN[t] = Lambda_IN[t] / (E_INp[t] + E_INm[t] + m)
    
print(f"{'t':>2} | {'ΔBM+':>7} | {'E_IN+':>7} | {'ΔIN+':>7} || "
      f"{'ΔBM-':>7} | {'E_IN-':>7} | {'ΔIN-':>7} || "
      f"{'Λ_IN':>8} | {'ρ_IN':>8}")
print("-" * 90)

for t in range(T):
    print(f"{t:2} | {delta_BMp[t]:7.2f} | {E_INp[t]:7.2f} | {delta_INp[t]:7.2f} || "
          f"{delta_BMm[t]:7.2f} | {E_INm[t]:7.2f} | {delta_INm[t]:7.2f} || "
          f"{Lambda_IN[t]:8.2f} | {rho_IN[t]:8.2f}")

 t |    ΔBM+ |   E_IN+ |    ΔIN+ ||    ΔBM- |   E_IN- |    ΔIN- ||     Λ_IN |     ρ_IN
------------------------------------------------------------------------------------------
 0 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 1 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 2 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 3 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 4 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 5 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 6 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 7 |    0.00 |    0.00 |    0.00 ||    0.00 |    0.00 |    0.00 ||     0.00 |     0.00
 8 |    0.00 |  -11.87 |    0.00 ||   11.87 |  -11.87 |    0.00 ||     0.00 |    -0.00
 9 |  -14.72 |    0.00 |    0.00 ||    

In [452]:
Lambda_b = np.zeros(T)
rho_b = np.zeros(T)
delta_bp = np.zeros(T)
delta_bm = np.zeros(T)
e_bp = np.zeros((I, T))
e_bm = np.zeros((I, T))

for t in range(T):
    delta_bp[t] = delta_BMp[t] - delta_INp[t]
    delta_bm[t] = delta_BMm[t] - delta_INm[t]
    Lambda_b[t] = (P_RT[t] - P_DA[t]) * delta_bp[t] - (P_PN[t] - P_DA[t]) * delta_bm[t]
    
    e_bp[:, t] = varphi_p[:, t] - e_INp[:, t]
    e_bm[:, t] = varphi_m[:, t] - e_INm[:, t]
    
    total_b = np.sum(e_bp[:, t] + e_bm[:, t]+m)

    if total_b < 1e-10:
        rho_b[t] = 0
    # Lambda_b[t]는 유지 (반영되지 않음)
    else:
        rho_b[t] = Lambda_b[t] / (total_b + m)
    
print(f"{'i':>2} {'t':>2} | {'φ⁺':>7} | {'e_IN⁺':>7} | {'e_b+':>9} || "
      f"{'φ⁻':>7} | {'e_IN⁻':>7} | {'e_b-':>9} || {'Λ_b':>10} | {'ρ_b':>8}")
print("-" * 120)

for i in range(I):
    for t in range(20, 22):
        print(f"{i:2} {t:2} | "
              f"{varphi_p[i, t]:7.4f} | {e_INp[i, t]:7.4f} | {e_bp[i, t]:9.4f} || "
              f"{varphi_m[i, t]:7.4f} | {e_INm[i, t]:7.4f} | {e_bm[i, t]:9.4f} || "
              f"{Lambda_b[t]:10.2f} | {rho_b[t]:8.2f}")

 i  t |      φ⁺ |   e_IN⁺ |      e_b+ ||      φ⁻ |   e_IN⁻ |      e_b- ||        Λ_b |      ρ_b
------------------------------------------------------------------------------------------------------------------------
 0 20 | 34.6195 |  3.8664 |   30.7531 ||  0.0000 |  0.0000 |    0.0000 ||    2544.91 |    33.09
 0 21 |  4.5623 |  0.4850 |    4.0774 ||  0.0000 |  0.0000 |    0.0000 ||     131.18 |    31.62
 1 20 |  3.2360 |  0.3614 |    2.8746 ||  0.0000 |  0.0000 |    0.0000 ||    2544.91 |    33.09
 1 21 |  0.0000 |  0.0000 |    0.0000 ||  0.0000 |  0.0000 |    0.0000 ||     131.18 |    31.62
 2 20 |  0.0000 |  0.0000 |    0.0000 ||  9.6688 |  9.6688 |    0.0000 ||    2544.91 |    33.09
 2 21 |  0.0000 |  0.0000 |    0.0000 ||  0.4934 |  0.4934 |    0.0000 ||     131.18 |    31.62
 3 20 | 43.2959 |  4.8354 |   38.4606 ||  0.0000 |  0.0000 |    0.0000 ||    2544.91 |    33.09
 3 21 |  0.0791 |  0.0084 |    0.0707 ||  0.0000 |  0.0000 |    0.0000 ||     131.18 |    31.62
 4 20 |  5.4230

In [453]:
lambda_it = np.zeros((I, T))
lambda_sum = np.zeros(T)

for t in range(T):
    for i in range(I):
        lambda_it[i, t] += rho_IN[t] * (e_INp[i, t] + e_INm[i, t])
        lambda_it[i, t] += rho_b[t] * (e_bp[i, t] + e_bm[i, t])
    lambda_sum[t] = np.sum(lambda_it[:, t])     

print(f"{'i':>2} {'t':>2} | {'λ_it':>10}")
print("-" * 20)
for t, i in product(range(20, 23), range(I)):
    print(f"{i:2} {t:2} | {lambda_it[i, t]:10.4f}")

 i  t |       λ_it
--------------------
 0 20 |   830.0561
 1 20 |    77.5876
 2 20 |  -469.1527
 3 20 |  1038.0876
 4 20 |   130.0250
 0 21 |   109.4803
 1 21 |     0.0000
 2 21 |   -19.8041
 3 21 |     1.8993
 4 21 |     0.0000
 0 22 |     0.0000
 1 22 |     0.0000
 2 22 |     0.0000
 3 22 |     0.0000
 4 22 |     0.0000


In [454]:
print(f"{'t':>2} | {'Λ_IN':>8} | {'Λ_b':>8} || {'∑λ_it':>10} | {'Difference':>10}")
print("-" * 65)

total_IN = 0
total_b  = 0
total_lambda = 0

for t in range(T):
    total_lhs = Lambda_IN[t] + Lambda_b[t]
    diff = total_lhs - lambda_sum[t]

    # 누적합 계산
    total_IN     += Lambda_IN[t]
    total_b      += Lambda_b[t]
    total_lambda += lambda_sum[t]

    print(f"{t:2} | {Lambda_IN[t]:8.2f} | {Lambda_b[t]:8.2f} || "
          f"{lambda_sum[t]:10.2f} | {diff:10.2f}")

# 마지막 줄: 총합 정리
print("-" * 65)
print(f"{'Σ':>2} | {total_IN:8.2f} | {total_b:8.2f}|| "
      f"{total_lambda:10.2f} | {total_IN + total_b - total_lambda:10.2f}")

 t |     Λ_IN |      Λ_b ||      ∑λ_it | Difference
-----------------------------------------------------------------
 0 |     0.00 |    -0.00 ||       0.00 |       0.00
 1 |     0.00 |     0.00 ||       0.00 |       0.00
 2 |     0.00 |    -0.00 ||       0.00 |       0.00
 3 |     0.00 |    -0.00 ||       0.00 |       0.00
 4 |     0.00 |    -0.00 ||       0.00 |       0.00
 5 |     0.00 |     0.00 ||       0.00 |       0.00
 6 |     0.00 |    -0.00 ||       0.00 |       0.00
 7 |     0.00 |     0.00 ||       0.00 |       0.00
 8 |     0.00 |  -673.28 ||    -673.28 |       0.00
 9 |     0.00 |   283.75 ||     283.75 |       0.00
10 | -2097.50 |  3913.94 ||    1816.44 |       0.00
11 |     0.00 | -2815.03 ||   -2815.03 |      -0.00
12 |     0.00 |     0.00 ||       0.00 |       0.00
13 |     0.00 |     0.00 ||       0.00 |       0.00
14 | -6902.90 | 23844.45 ||   16941.55 |       0.00
15 |     0.00 |     0.00 ||       0.00 |       0.00
16 | -5897.21 | 18545.88 ||   12648.67 |       0.0